Author: Eklavya Gupta

Importing

In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from tf_emb1 import TensorflowHubEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers
from langchain_community.vectorstores import Typesense
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore


DATA_PATH = "Docs/"
# USE_PATH = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
USE_PATH = "D:/Models/Sentence Embeddings/universal sentence encoder"

embedding = TensorflowHubEmbeddings(model_url=USE_PATH)
embedding.model_url = USE_PATH
embedding.load()

C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


loading and chunking using Direct Preference Optimization research paper

In [2]:
loader = PyPDFLoader("PDF/DPO.pdf")
pages = loader.load()
print(len(pages))
splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap=90)
docs = splitter.split_documents(pages)
print(len(docs))

27
409


adding data to pinecone database

In [3]:
from langchain_pinecone import PineconeVectorStore
from langchain_pinecone import Pinecone
pinecone_api_key="0245dc9d-e4f7-49a5-8221-cc3d6e4c975f"
index_name="lang"
namespace="vecc"
vectorstore = Pinecone(
    index_name=index_name,
    embedding=embedding,
    namespace=namespace,
    pinecone_api_key=pinecone_api_key
)
vectorstore.add_texts([doc.page_content for doc in docs])

C:\Users\EKLAVYA\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.3 and will be removed in 0.3.0. Use PineconeVectorStore instead.
  warn_deprecated(


['0729c382-583b-4157-a9b6-19ed4acd77af',
 'da10d098-0cc8-4200-bb99-ed320f9c8e96',
 '160f53f5-0004-4374-b2a2-31d2fd188d06',
 'eabe4717-e688-4b13-b54e-7ff6f91590a9',
 'a0572420-b92c-4d68-a66d-cf2d9c255c9b',
 '3f174a60-7121-462e-ba1d-854301d8dbc6',
 '683c1c56-4649-4bba-9d34-603f7d9242d7',
 'c8c3aceb-6676-4f77-818c-5617ffb0ad83',
 '6c150d7d-8aae-447d-9eaa-e9c4da1c3c52',
 'f1c2bea0-8243-4ed6-99c1-098f05b5d37a',
 'f38a6f54-e98f-4e74-aa75-968e47302025',
 'c3581ef4-2022-46cc-ac2b-47434b0606ac',
 '5cbac165-2b6d-45ab-9013-4b023d1ca8c6',
 'bb108485-2457-4ca7-a6bd-7857ca16d321',
 '53b0472f-20c4-4c91-bd38-37a0961fa11b',
 '90e6a12f-f8aa-4201-9cde-7218d10ac38d',
 'e2ff0dc0-4447-46c5-b2d7-28539d7b22fc',
 '729fed96-06b5-468d-be10-ddd7715aea07',
 '972d141d-b9e9-4a10-a12c-dc5d3d8bb49f',
 '96d6e4f3-43bc-4550-bfe1-3794b62ff10c',
 '24f97ef8-6ee7-4cb3-a192-93268e2037f6',
 'ce336616-c785-4d02-9941-37a79d53ede2',
 '98fc36de-29cd-46ab-9930-31fc994ae38c',
 '54a01a06-de3d-4ebf-bcfe-c2dad066d8af',
 'b408dafd-938b-

using rapid api and gpt4 and some preprocessings

In [37]:
def response(query):
    prompt_template = """Context: {context} Question: {question} Only returns the helpful answer below and nothing else. Helpful answer:"""

    system_prompt = '''Use the following pieces of information to answer the user's question. If you don't know the answer, please just say that you don't know the answer, don't try to make up an answer.'''

    context = vectorstore.similarity_search(query,k=4)
    context = "\n".join([doc.page_content for doc in context])

    prompt_template = prompt_template.replace("{context}", context)
    prompt_template = prompt_template.replace("{question}", query)

    import http.client

    conn = http.client.HTTPSConnection("chatgpt-42.p.rapidapi.com")

    payload = "{\"messages\":[{\"role\":\"user\",\"content\":\"{prompt_template}\"}],\"system_prompt\":\"{system_prompt}\",\"temperature\":0.4,\"top_k\":5,\"top_p\":0.4,\"max_tokens\":512,\"web_access\":false}"

    headers = {
        'x-rapidapi-key': "ff0f9e7c95msh334e99eb525cc2cp195eb8jsn5b22ce2e1275",
        'x-rapidapi-host': "chatgpt-42.p.rapidapi.com",
        'Content-Type': "application/json"
    }
    

    payload = payload.replace("{prompt_template}", prompt_template)
    payload = payload.replace("{system_prompt}", system_prompt)

    cnt = payload.count("\n")
    for i in range(cnt):
        payload = payload.replace("\n"," ")

    print(payload)

    conn.request("POST", "/conversationgpt4-2", payload, headers)

    res = conn.getresponse()
    data = res.read()

    return str(data.decode("utf-8"))

enjoying good results

In [38]:
query="what is direct preference optimization"
result = response(query)

{"messages":[{"role":"user","content":"Context: model responses, DPO can therefore optimize a policy using a simple binary cross entropy objective, producing the optimal policy to an implicit reward function fit to the preference data. Our main contribution is Direct Preference Optimization (DPO), a simple RL-free algorithm for such as fine-tuning language models, our goal is to derive a simple approach for policy optimization using preferences directly. Unlike prior RLHF methods, which learn a reward and then optimize it via RL, our approach leverages a particular choice of reward model parameterization that enables In this paper, we show how to directly optimize a language model to adhere to human preferences, without explicit reward modeling or reinforcement learning. We propose Direct Preference Optimiza- tion (DPO) , an algorithm that implicitly optimizes the same objective as existing RLHF algorithms objective, fitting an implicit reward model whose corresponding optimal policy c

In [39]:
print(result)

{"result":"Direct Preference Optimization (DPO) is a Reinforcement Learning (RL)-free algorithm designed specifically for tasks like fine-tuning language models based on human preferences. It aims at deriving a straightforward methodology for policy optimization utilizing these preferences directly rather than through traditional means involving explicit rewards modelling or reinforcement learning techniques. The key advantage lies in its ability to leverage specific choices regarding reward model parameters enabling efficient extraction of the best possible policies from fitted implicit reward functions derived solely from available preference data. This makes DPO particularly useful when dealing with complex scenarios where conventional approaches might struggle due to their reliance upon explicitly defined objectives or extensive training requirements associated with standard RL paradigms. \n  \nIn summary, Direct Preference Optimization offers a novel way forward within AI research